In [ ]:
!pip install timm
!pip install --upgrade wandb

     |████████████████████████████████| 431 kB 30.5 MB/s 
     |████████████████████████████████| 1.8 MB 9.2 MB/s 
     |████████████████████████████████| 181 kB 49.6 MB/s 
     |████████████████████████████████| 144 kB 47.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=983e4d3ce4a5e98712be7ab7b1061a5c4b4442139df4b17b6d2b4b2b68a8d3d3
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 26.1 MB/s 
     |████████████████████████████████| 948 kB 56.0 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=dd1cda5dbfc7e39213ddcf050d8872bbcc6f1d5e175476c7df4e54f9727167f4
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:

import albumentations 
from albumentations.pytorch import ToTensorV2

In [ ]:
!pip install colorama

In [ ]:
import os
import gc
import cv2
import math
import copy
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import wandb
# Log in to your W&B account
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
CONFIG = {"seed": 2022,
          "epochs": 25,
          "img_size": 448,
          "model_name": "tf_efficientnet_b0_ns",
          "num_classes": 100,
          "embedding_size": 512,
          "train_batch_size": 32,
          "valid_batch_size": 64,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          # ArcFace Hyperparameters
          "s": 30.0, 
          "m": 0.50,
          "ls_eps": 0.0,
          "easy_margin": False
          }

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/dm-turtle-recall/train.csv \
    -O /content/train.csv

!wget --no-check-certificate \
    https://storage.googleapis.com/dm-turtle-recall/extra_images.csv \
    -O /content/extra_images.csv

--2022-04-14 01:52:01--  https://storage.googleapis.com/dm-turtle-recall/train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.112, 142.250.188.48, 172.217.13.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66538 (65K) [text/csv]
Saving to: ‘/content/train.csv’

/content/train.csv  100%[===================>]  64.98K  --.-KB/s    in 0.001s  

2022-04-14 01:52:01 (91.5 MB/s) - ‘/content/train.csv’ saved [66538/66538]

--2022-04-14 01:52:01--  https://storage.googleapis.com/dm-turtle-recall/extra_images.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.15.112, 142.250.81.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277127 (271K) [text/csv]
Saving to: ‘/content/extra_images.csv’

/content/extra_imag 10

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/dm-turtle-recall/test.csv \
    -O /content/test.csv

--2022-04-14 01:52:01--  https://storage.googleapis.com/dm-turtle-recall/test.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.81.208, 142.250.188.48, 142.251.45.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.81.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8356 (8.2K) [text/csv]
Saving to: ‘/content/test.csv’

/content/test.csv   100%[===================>]   8.16K  --.-KB/s    in 0s      

2022-04-14 01:52:01 (23.1 MB/s) - ‘/content/test.csv’ saved [8356/8356]



In [ ]:
!mkdir Img
!cd /content/Img
!wget --no-check-certificate \
    https://storage.googleapis.com/dm-turtle-recall/images.tar \
    -O /content/Img/images.tar
!tar -xvf /content/Img/images.tar -C /content/Img/

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
./ID_I61KEZYE.JPG
./ID_908ERZ44.JPG
./ID_JLYCG485.JPG
./ID_448G8JTW.JPG
./ID_A258JDE5.JPG
./ID_E1DR2CEG.JPG
./ID_VM575DNC.JPG
./ID_FIB5ACGK.JPG
./ID_TX2I1627.JPG
./ID_3JJCODLR.JPG
./ID_WZINZNDB.JPG
./ID_JDTAKGNN.JPG
./ID_LRN9UTHC.JPG
./ID_DVLMJHII.JPG
./ID_GDUNI351.JPG
./ID_GY0AKVTX.JPG
./ID_FZ4NBOR0.JPG
./ID_1CRQFLPQ.JPG
./ID_URTL0SBO.JPG
./ID_OI3029CM.JPG
./ID_FQ5K7KB5.JPG
./ID_9O4CE246.JPG
./ID_OEE32CZ4.JPG
./ID_RN1GNW24.JPG
./ID_CGAA7VNO.JPG
./ID_WM7CUJXW.JPG
./ID_0X0OFK1L.JPG
./ID_VUF915T5.JPG
./ID_LRLI49AW.JPG
./ID_BW3I3GVJ.JPG
./ID_D9GAQD0V.JPG
./ID_O4N08V2Y.JPG
./ID_6VVFBZ7G.JPG
./ID_EV9KWF0K.JPG
./ID_K0Y4VFNU.JPG
./ID_RM3V8OAZ.JPG
./ID_B5CETEGU.JPG
./ID_OYP4IYCU.JPG
./ID_TSA11E82.JPG
./ID_YQLDQK1F.JPG
./ID_4ROWHFUG.JPG
./ID_W56CAWNC.JPG
./ID_TB3TLP3D.JPG
./ID_BI4UPWUT.JPG
./ID_ZWUAGZ0H.JPG
./ID_WBR9VW0J.JPG
./ID_GAJY8HFS.JPG
./ID_Q3GPYWHV.JPG
./ID_CKSNSCJC.JPG
./ID_IZVXC8V9.JPG
./ID_8MQTSPY0.JPG
./ID

In [ ]:
!rm /content/Img/images.tar

In [ ]:
import pandas as pd

train=pd.read_csv('/content/train.csv')
train

,image_id,image_location,turtle_id
0,ID_2RK4WLN8,top,t_id_VP2NW7aV
1,ID_VVW0QXLX,left,t_id_qZ0iZYsC
2,ID_RVATH2HZ,Right,t_id_3b65X5Lw
3,ID_2GB90GPS,left,t_id_YjXYTCGC
4,ID_LM6S0B1M,top,t_id_d6aYXtor
...,...,...,...
2140,ID_BDMVQH6G,left,t_id_SwQZGIpa
2141,ID_JD58AF27,left,t_id_QqeoI5F3
2142,ID_Y2LJOVUQ,left,t_id_Lhp87PBX
2143,ID_GH1RFB6Z,right,t_id_FBsGDJhU


In [ ]:
ROOT_DIR = '/content/'
TRAIN_DIR = '/content/Img'
TEST_DIR = '/content/Img'

In [ ]:
def get_train_file_path(id):
    return f"{TRAIN_DIR}/{id}"+".JPG"

In [ ]:
df = pd.read_csv(f"{ROOT_DIR}/train.csv")
df['file_path'] = df['image_id'].apply(get_train_file_path)
df.head()

,image_id,image_location,turtle_id,file_path
0,ID_2RK4WLN8,top,t_id_VP2NW7aV,/content/Img/ID_2RK4WLN8.JPG
1,ID_VVW0QXLX,left,t_id_qZ0iZYsC,/content/Img/ID_VVW0QXLX.JPG
2,ID_RVATH2HZ,Right,t_id_3b65X5Lw,/content/Img/ID_RVATH2HZ.JPG
3,ID_2GB90GPS,left,t_id_YjXYTCGC,/content/Img/ID_2GB90GPS.JPG
4,ID_LM6S0B1M,top,t_id_d6aYXtor,/content/Img/ID_LM6S0B1M.JPG


In [ ]:
encoder = LabelEncoder()
df['turtle_id'] = encoder.fit_transform(df['turtle_id'])

with open("le.pkl", "wb") as fp:
    joblib.dump(encoder, fp)

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.turtle_id)):
      df.loc[val_ , "kfold"] = fold

In [ ]:
class HappyWhaleDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['turtle_id'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, 
                 m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CONFIG['device'])
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [ ]:
class HappyWhaleModel(nn.Module):
    def __init__(self, model_name, embedding_size, pretrained=True):
        super(HappyWhaleModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.embedding = nn.Linear(in_features, embedding_size)
        self.fc = ArcMarginProduct(embedding_size, 
                                   CONFIG["num_classes"],
                                   s=CONFIG["s"], 
                                   m=CONFIG["m"], 
                                   easy_margin=CONFIG["ls_eps"], 
                                   ls_eps=CONFIG["ls_eps"])

    def forward(self, images, labels):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        embedding = self.embedding(pooled_features)
        output = self.fc(embedding, labels)
        return output
    
    def extract(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        embedding = self.embedding(pooled_features)
        return embedding

    
model = HappyWhaleModel(CONFIG['model_name'], CONFIG['embedding_size'])
model.to(CONFIG['device']);

In [ ]:
def criterion(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        outputs = model(images, labels)
        loss = criterion(outputs, labels)
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.long)
        
        batch_size = images.size(0)

        outputs = model(images, labels)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "Loss{:.4f}_epoch{:.0f}.bin".format(best_epoch_loss, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = HappyWhaleDataset(df_train, transforms=data_transforms["train"])
    valid_dataset = HappyWhaleDataset(df_valid, transforms=data_transforms["valid"])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
train_loader, valid_loader = prepare_loaders(df, fold=0)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                       weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

In [ ]:
run = wandb.init(project='HappyWhale', 
                 config=CONFIG,
                 job_type='Train',
                 tags=['arcface', 'gem-pooling', 'effnet-b0-ns', '448'],
                 anonymous='must')

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   55977 KB |   55977 KB |   56158 KB |  185344 B  |\n|       from large pool |   16800 KB |   16800 KB |   16800 KB |       0 B  |\n|       from small pool |   39177 KB |   39177 KB |   39358 KB |  185344 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |   55977 KB |   55977 KB |   56158 KB |  185344 B  |\n|       from large pool |   16800 KB |   16800 KB |

In [ ]:
import gc
gc.collect()

493

In [ ]:
model, history = run_training(model, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=50)

[INFO] Using GPU: Tesla T4



100%|██████████| 7/7 [00:13<00:00,  1.89s/it, Epoch=1, LR=1.48e-5, Valid_Loss=13.6]


Validation Loss Improved (inf ---> 13.612734367797424)
Model Saved



100%|██████████| 7/7 [00:13<00:00,  1.91s/it, Epoch=2, LR=5.58e-6, Valid_Loss=13.6]


Validation Loss Improved (13.612734367797424 ---> 13.551028396144059)
Model Saved



...

100%|██████████| 7/7 [00:12<00:00,  1.84s/it, Epoch=47, LR=7.04e-5, Valid_Loss=10.7]


100%|██████████| 7/7 [00:13<00:00,  1.86s/it, Epoch=48, LR=8.42e-5, Valid_Loss=10.6]


100%|██████████| 7/7 [00:13<00:00,  1.86s/it, Epoch=49, LR=9.42e-5, Valid_Loss=10.8]


100%|██████████| 7/7 [00:13<00:00,  1.86s/it, Epoch=50, LR=9.94e-5, Valid_Loss=10.6]



Training complete in 1h 10m 4s
Best Loss: 10.3597


In [ ]:
run.finish()

Train Loss,███▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▄▃▃▂▂▁▁
Valid Loss,██▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▅▄▃▃▂▂▁▁
Best Loss,13.71171
Train Loss,5.84367
Valid Loss,13.71171
